In [5]:
! rm craigslist.csv; scrapy crawl craigslist --nolog -o craigslist.csv -t csv

In [1]:
! rm kijiji.csv; scrapy crawl kijiji --nolog -o kijiji.csv -t csv

rm: cannot remove 'kijiji.csv': No such file or directory


In [2]:
import pandas as pd

craigslist = pd.read_csv('craigslist.csv')
kijiji = pd.read_csv('kijiji.csv')

df = craigslist.append(kijiji, ignore_index=True)

In [3]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

In [4]:
commercial_drive_coord = 49.263277, -123.068839
df2 = df.assign(f = (df['lat'] - commercial_drive_coord[0])**2 + (df['long'] - commercial_drive_coord[1])**2).sort_values('f').drop('f', axis=1)
df2 = df2[df2['furnished']==True]
df2.style.format({'url': make_clickable})

,url,price,lat,long,beds,area,furnished,available
2504,https://vancouver.craigslist.org/van/apa/d/vancouver-commercial-drive-garden-suite/6876968515.html,1400,49.2651,-123.066,nan,500,True,jun 1
2948,https://vancouver.craigslist.org/van/apa/d/vancouver-options-to-choose-1bdrmpatio/6877182138.html,1275,49.2622,-123.072,1,nan,True,nan
4055,https://www.kijiji.ca/v-room-rental-roommate/vancouver/895-beautiful-room-student-or-traveler-mountain-view/1425185452,895,49.2585,-123.072,nan,nan,True,nan
3233,https://www.kijiji.ca/v-room-rental-roommate/vancouver/small-semi-furn-room-female-student-broadway-vcc-skytrain/1429535542,395,49.2607,-123.075,nan,nan,True,nan
4621,https://www.kijiji.ca/v-room-rental-roommate/vancouver/near-downtown-vcc-broadway-skytrain-furn-room-female-student/1429747062,579,49.2607,-123.075,nan,nan,True,nan
3117,https://www.kijiji.ca/v-room-rental-roommate/vancouver/downtown-broadway-skytrn-fem-student-wrkvisa-priv-furn-rm/1429995366,629,49.2607,-123.075,nan,nan,True,nan
4417,https://www.kijiji.ca/v-room-rental-roommate/vancouver/ensuite-bedroom-in-townhouse-near-clark-and-broadway/1428184339,1050,49.2632,-123.077,3,nan,True,nan
3549,https://www.kijiji.ca/v-room-rental-roommate/vancouver/international-students-work-holiday-house-commercial-dr/1428085027,550,49.2625,-123.06,nan,nan,True,nan
1171,https://vancouver.craigslist.org/van/apa/d/vancouver-furnished-2-bedrooms/6876088633.html,2900,49.267,-123.077,2,1000,True,apr 30
4342,https://www.kijiji.ca/v-apartments-condos/vancouver/commercial-drive-rental-2-bed-1-bath-live-near-the-drive/1427617294,2500,49.2707,-123.075,2,nan,True,nan
